In [2]:
from pathlib import Path
import pandas as pd

BASE_DIR = Path.cwd()

# Projeyi yanlış klasörden açtıysan düzeltmek için:
# Eğer notebooks klasöründeysen, proje root = bir üst klasör olabilir.
# Aşağıdaki satır çoğu projede işe yarar:
if (BASE_DIR / "data").exists() is False and (BASE_DIR.parent / "data").exists():
    BASE_DIR = BASE_DIR.parent

PROCESSED_DIR = BASE_DIR / "data" / "processed"

candidates = sorted(PROCESSED_DIR.glob("districts_features*.csv")) + sorted(PROCESSED_DIR.glob("districts_*final*.csv"))
print("BASE_DIR:", BASE_DIR)
print("PROCESSED_DIR:", PROCESSED_DIR)
print("Bulunan aday dosyalar:")
for p in candidates[:20]:
    print(" -", p.name)

# En mantıklı dosyayı seç: daha önce sende görünen buydu
preferred = PROCESSED_DIR / "districts_features_v3_predictions.csv"
features_path = preferred if preferred.exists() else (candidates[0] if candidates else None)

if features_path is None:
    raise FileNotFoundError(f"Processed klasöründe CSV bulamadım: {PROCESSED_DIR}")

print("\nKullanılan dosya:", features_path)

df = pd.read_csv(features_path)
print("Satır:", len(df), "Kolon:", df.shape[1])
print("Kolonlar:", df.columns.tolist())
df.head()


BASE_DIR: C:\Kodlar\greensense-ai
PROCESSED_DIR: C:\Kodlar\greensense-ai\data\processed
Bulunan aday dosyalar:
 - districts_features_v1.csv
 - districts_features_v2_model.csv
 - districts_features_v3_predictions.csv
 - districts_features_v4_pred_treecover.csv
 - districts_trees_needed_final.csv

Kullanılan dosya: C:\Kodlar\greensense-ai\data\processed\districts_features_v3_predictions.csv
Satır: 929 Kolon: 13
Kolonlar: ['province_name', 'district_name', 'lat', 'lon', 'avg_temp', 'avg_rain', 'treecover_pct', 'potential_treecover_pct', 'missing_treecover_pct', 'model_potential_treecover_pct', 'model_missing_treecover_pct', 'pred_treecover_pct', 'treecover_gap_pct']


,province_name,district_name,lat,lon,avg_temp,avg_rain,treecover_pct,potential_treecover_pct,missing_treecover_pct,model_potential_treecover_pct,model_missing_treecover_pct,pred_treecover_pct,treecover_gap_pct
0,Adana,Aladağ,37.666642,35.387781,16.739615,0.0,54.58,10,0.00,44.543306,0.000000,45.610237,0.000000
1,Adana,Ceyhan,37.011888,35.768198,19.804077,0.0,0.05,10,9.95,27.848537,27.798537,37.492598,37.442598
2,Adana,Feke,37.871495,35.821754,10.716615,0.0,15.07,10,0.00,30.322210,15.252210,27.646413,12.576413
3,Adana,Karaisali,37.259147,35.142888,12.239538,0.0,8.33,10,1.67,16.481983,8.151983,18.054020,9.724020
4,Adana,Karataş,36.675979,35.229132,21.450077,0.0,0.00,5,5.00,3.265905,3.265905,3.663069,3.663069


In [3]:
import numpy as np

col = "treecover_pct"
if col not in df.columns:
    raise KeyError(f"{col} kolonu yok. Olan kolonlar: {df.columns.tolist()}")

print("treecover_pct describe:\n", df[col].describe())
print("\n0 oranı:", (df[col] == 0).mean())

mx, mn = df[col].max(), df[col].min()
print("\nmin,max:", mn, mx)

# Ölçek kontrolü
if mx <= 1.0:
    print("\n⚠️ treecover_pct 0-1 ölçeğinde görünüyor. % için *100 yapılmalı.")
else:
    print("\n✅ treecover_pct 0-100 ölçeğinde görünüyor.")

# Antalya ilçeleri örnek
mask_ant = df["province_name"].astype(str).str.lower().str.strip().eq("antalya")
if mask_ant.any():
    ant = df.loc[mask_ant, ["district_name", col]].sort_values(col)
    print("\nAntalya en düşük 20:\n", ant.head(20))
    print("\nAntalya en yüksek 20:\n", ant.tail(20))
else:
    print("\n⚠️ province_name içinde 'Antalya' yok (yazım farklı olabilir). Benzeyenleri:")
    print(df["province_name"].astype(str).str.lower().value_counts().head(20))


treecover_pct describe:
 count    929.000000
mean      13.205587
std       25.567973
min        0.000000
25%        0.000000
50%        0.000000
75%       11.100000
max       98.720000
Name: treecover_pct, dtype: float64

0 oranı: 0.5694294940796556

min,max: 0.0 98.72

✅ treecover_pct 0-100 ölçeğinde görünüyor.

Antalya en düşük 20:
     district_name  treecover_pct
86         Akseki           0.00
88         Elmalı           0.00
89         Finike           0.00
95      Korkuteli           0.00
100        İbradi           0.00
98         Merkez           0.62
93            Kaş          11.96
87         Alanya          12.96
99          Serik          16.90
96        Kumluca          24.42
97       Manavgat          36.43
92           Kale          38.40
94          Kemer          38.91
91      Gündoğmuş          42.22
90       Gazipaşa          47.36

Antalya en yüksek 20:
     district_name  treecover_pct
86         Akseki           0.00
88         Elmalı           0.00
89         F

In [4]:
import numpy as np

df2 = df.copy()

# 0'ları "muhtemelen veri yok" olarak işaretle
df2["treecover_pct_fixed"] = df2["treecover_pct"].replace(0, np.nan)

df2["has_treecover_data"] = df2["treecover_pct_fixed"].notna()
